In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
dataPath = "..\\Extraction\\pixels.tsv" 

pixels = []
with open(dataPath, "r") as pixel_data:
    line = pixel_data.readline()
    while line != "":
        tokens = line.split("\t")
        pixels.append((int(tokens[0]), int(tokens[1]), int(tokens[2])))
        line = pixel_data.readline()

total_pixels = len(pixels)
print ("Number of pixels : " + str(total_pixels))

avg_over = int(total_pixels / 1080)
print ("Avg Over : " + str(avg_over))
reduced_set = []
colour_set = []
count = 0
current_value = 0;
current_colour = (0, 0, 0)
max_value = 0;
for pixel in pixels:
    if count == avg_over:
        amp_val = int(current_value / avg_over)
        reduced_set.append(amp_val)
        if(amp_val > max_value):
            max_value = amp_val
        current_value = 0
        current_colour = (
            int(current_colour[0] / avg_over), 
            int(current_colour[1] / avg_over),
            int(current_colour[2] / avg_over))
        colour_set.append(current_colour)
        current_colour = (0, 0, 0)
        count = 0
    else:
        count += 1
        grayScale = int(((pixel[0] * 0.3) + (pixel[1] * 0.59) + (pixel[2] * 0.11)))
        current_value += grayScale
        current_colour = (current_colour[0] + pixel[0], current_colour[1] + pixel[1], current_colour[2] + pixel[2])
        
print ("reduced_set count : " + str(len(reduced_set)))
#print (str(reduced_set))
print ("colour_set count : " + str(len(colour_set)))
#print (str(colour_set))
print ("Max Amp Value : " + str(max_value))

In [ ]:
center = 50
maxY = 256 + center
normaliseMult = 256.0 / float(max_value)

normalisedSet = []
for raw in reduced_set:
    normalisedSet.append(int(float(raw) * normaliseMult))

#data = np.array(reduced_set)
data = np.array(normalisedSet)

N = len(data)
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
radii = data
width = 2 * np.pi / N

ax = plt.subplot(111, polar=True)
ax.grid(visible=False)
ax.spines['polar'].set_visible(False)

# render a line to smooth the tops of the bars
add_smother = True
if add_smother:
    radiiLine = []
    for i in data:
        radiiLine.append(i + 50)
    ax.plot(theta, radiiLine, color='black', linewidth=3)

bars = ax.bar(theta, radii, width=width, bottom=center)
ax.yaxis.set_ticks([])
ax.xaxis.set_ticks([])

index = 0.0
count = 0
for r, bar in zip(theta, bars):
    myColor = (colour_set[count][0] / 256.0,
               colour_set[count][1] / 256.0,
               colour_set[count][2] / 256.0)
    bar.set_facecolor(myColor)
    bar.set_edgecolor(myColor)
    bar.set_alpha(0.8)
    count += 1
    
fig_size = plt.rcParams["figure.figsize"]
#fig_size[0] = 30
#fig_size[1] = 15
plt.rcParams["figure.figsize"] = fig_size
plt.ylim((0, maxY))
plt.savefig('..\\movie_polar.png')
plt.show()